Movie Review Dataset


Well start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already
preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the
entire dataset. For example a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.


In [40]:
from keras.datasets import imdb
from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584 #Number of words in the dataset 
MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data (num_words = VOCAB_SIZE)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [41]:
#let's look at the review 
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

More Preprocessing

If we have a look at some of our loaded in reviews we'll notice that they are different lengths. This is an issue. We cannot pass different length
data into our neural network. Therefore we must make each review the same length. To do this we will follow the procedure below:
• if the review is greater than 250 words then trim off the extra words
• if the review is less than 250 words add the necessary amount of's to make it equal to 250.
Luckily for us keras has a function that can do this for us:


In [42]:

train_data = pad_sequences(train_data, MAXLEN)
test_data = pad_sequences(test_data, MAXLEN)

In [43]:
len(train_data[0])

250

Creating the Model


Now it's time to create the model. We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds
into a dense node to get our predicted sentiment.
32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!


In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding (VOCAB_SIZE, 32), # every single word has 32 dimention
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [34]:
#Training the model
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 58s 90ms/step - loss: 0.4129 - acc: 0.8108 - val_loss: 0.2804 - val_acc: 0.8840
Epoch 2/10
625/625 [==============================] - 56s 90ms/step - loss: 0.2384 - acc: 0.9090 - val_loss: 0.2828 - val_acc: 0.8796
Epoch 3/10
625/625 [==============================] - 55s 89ms/step - loss: 0.1848 - acc: 0.9341 - val_loss: 0.2778 - val_acc: 0.8790
Epoch 4/10
625/625 [==============================] - 54s 86ms/step - loss: 0.1531 - acc: 0.9473 - val_loss: 0.2787 - val_acc: 0.8920
Epoch 5/10
625/625 [==============================] - 60s 95ms/step - loss: 0.1308 - acc: 0.9541 - val_loss: 0.2878 - val_acc: 0.8856
Epoch 6/10
625/625 [==============================] - 48s 77ms/step - loss: 0.1134 - acc: 0.9614 - val_loss: 0.3418 - val_acc: 0.8934
Epoch 7/10
625/625 [==============================] - 51s 82ms/step - loss: 0.0976 - acc: 0.9672 - val_loss: 0.3815 - val_acc: 0.8840
Epoch 8/10
625/625 [==============================] - 50s 80ms

In [35]:
#Evaluating the model
results = model.evaluate(test_data, test_labels)
print(results )

782/782 [==============================] - 17s 21ms/step - loss: 0.4729 - acc: 0.8555
[0.47288480401039124, 0.8555200099945068]


So were scoring somewhere in the mid-high 80's. Not bad for a simple recurrent network.


- Making Predictions
Now lets use our network to make predictions on our own reviews.
since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well
load the encodings from the dataset and use them to encode our own data.


In [47]:

import keras
word_index = imdb.get_word_index()
def encode_text (text) :
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return pad_sequences([tokens], MAXLEN)[0]

text ="that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [54]:
# while were at it lets make a decode function
reverse_word_index = {value: key for (key, value) in word_index.items()}
def decode_integers(integers):
                      PAD = 0
                      text = ""
                      for num in integers:
                        if num != PAD:
                          text += reverse_word_index[num] + " "
                      
                      return text[:-1]
print (decode_integers(encoded))


that movie was just amazing so amazing


In [72]:
# now time to make a prediction
def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1, 250))
    pred[0] = encoded_text
    result = model.predict(pred)
    print(result[0])
    
    
positive_review = "That movie was so awesome! I really loved it and would watch it again because it was amazingly great"
predict(positive_review)
negative_review = "that movie sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict (negative_review)



1/1 [==============================] - 0s 203ms/step
[0.8535881]
1/1 [==============================] - 0s 52ms/step
[0.45796007]
